In [ ]:
!nvidia-smi

Tue Jul  6 21:10:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers==4.5.0
!pip install pytorch.lightning==1.2.7

     |████████████████████████████████| 2.2MB 7.8MB/s 
     |████████████████████████████████| 901kB 35.8MB/s 
     |████████████████████████████████| 3.3MB 48.1MB/s 
     |████████████████████████████████| 839kB 8.0MB/s 
     |████████████████████████████████| 829kB 47.6MB/s 
     |████████████████████████████████| 276kB 46.2MB/s 
     |████████████████████████████████| 235kB 45.3MB/s 
     |████████████████████████████████| 122kB 56.5MB/s 
     |████████████████████████████████| 1.3MB 49.8MB/s 
     |████████████████████████████████| 143kB 55.3MB/s 
     |████████████████████████████████| 296kB 51.1MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=2f7f9efd1274c99621c9f7dc359ccb6bd5b02328150620602aff773dc73897d7
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44636 sha256=d5c3b8402786044335d61cd5b0eb4a

In [ ]:
import json
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap

from transformers import (T5ForConditionalGeneration,
                          AdamW)

from transformers import T5TokenizerFast as T5Tokenizer

from tqdm.auto import tqdm

In [ ]:
!unzip "/content/archive (3).zip"

Archive:  /content/archive (3).zip
  inflating: news_summary.csv        
  inflating: news_summary_more.csv   


In [ ]:
df = pd.read_csv("/content/news_summary.csv", encoding="latin-1")

df

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
...,...,...,...,...,...,...
4509,Mansha Mahajan,"24 Feb 2017,Friday",Rasna seeking ?250 cr revenue from snack categ...,http://indiatoday.intoday.in/story/rasna-eyes-...,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m..."
4510,Dishant Sharma,"03 Aug 2017,Thursday",Sachin attends Rajya Sabha after questions on ...,http://indiatoday.intoday.in/story/sachin-tend...,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...
4511,Tanya Dhingra,"03 Aug 2017,Thursday",Shouldn't rob their childhood: Aamir on kids r...,http://www.hindustantimes.com/bollywood/secret...,"Aamir Khan, while talking about reality shows ...","Aamir Khan, whose last film Dangal told the st..."
4512,Pragya Swastik,"07 Dec 2016,Wednesday","Asha Bhosle gets ?53,000 power bill for unused...",http://indiatoday.intoday.in/story/singer-asha...,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...


In [ ]:
df = pd.DataFrame(data=(df["text"], df["ctext"]))

In [ ]:
df = df.T

In [ ]:
df

,text,ctext
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
...,...,...
4509,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m..."
4510,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...
4511,"Aamir Khan, while talking about reality shows ...","Aamir Khan, whose last film Dangal told the st..."
4512,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...


In [ ]:
df.columns = ["summary", "text"]

In [ ]:
df = df.dropna()

In [ ]:
df

,summary,text
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
...,...,...
4509,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m..."
4510,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...
4511,"Aamir Khan, while talking about reality shows ...","Aamir Khan, whose last film Dangal told the st..."
4512,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.15)

train_df.shape, test_df.shape

((3736, 2), (660, 2))

In [ ]:
class SummaryDataset(Dataset):

    def __init__(self, 
                 data,
                 tokenizer,
                 text_max_tok_len=512,
                 summary_max_tok_len=128):
        
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_tok_len = text_max_tok_len
        self.summary_max_tok_len = summary_max_tok_len

    def __len__(self):

        return len(self.data)

    def __getitem__(self, index):

        data_row = self.data.iloc[index]

        src_text = data_row["text"]

        text_encoding = self.tokenizer(
            src_text,
            max_length=self.text_max_tok_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        summary = data_row["summary"]

        summary_encoding = self.tokenizer(
            summary,
            max_length=self.summary_max_tok_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        labels = summary_encoding["input_ids"]
        labels[labels == 0] = -100

        return dict(text=src_text,
                    summary=data_row["summary"],
                    text_input_ids=text_encoding["input_ids"].flatten(),
                    text_attention_mask=text_encoding["attention_mask"].flatten(),
                    labels=labels.flatten(),
                    labels_attention_mask=summary_encoding["attention_mask"].flatten())
        


In [ ]:
class SummaryDataModule(pl.LightningDataModule):

    def __init__(self, 
                 train_df,
                 test_df,
                 tokenizer,
                 batch_size=8,
                 text_max_tok_len=512,
                 summary_max_tok_len=128):
        
        super().__init__()

        self.train_df = train_df
        self.test_df = test_df

        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.text_max_tok_len = text_max_tok_len
        self.summary_max_tok_len = summary_max_tok_len

    def setup(self, stage=None):

        self.train_dataset = SummaryDataset(self.train_df,
                                            self.tokenizer,
                                            self.text_max_tok_len,
                                            self.summary_max_tok_len)
        
        self.test_dataset = SummaryDataset(self.test_df,
                                            self.tokenizer,
                                            self.text_max_tok_len,
                                            self.summary_max_tok_len)
        
    def train_dataloader(self):

        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          shuffle=True,
                          num_workers=2)
        
    def val_dataloader(self):

        return DataLoader(self.test_dataset,
                          batch_size=self.batch_size,
                          shuffle=False,
                          num_workers=2)
        
    def test_dataloader(self):

        return DataLoader(self.test_dataset,
                          batch_size=self.batch_size,
                          shuffle=False,
                          num_workers=2)
        



In [ ]:
MODEL_NAME = "t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)


N_EPOCHS = 6
BATCH_SIZE = 8
data_module = SummaryDataModule(train_df, test_df, tokenizer, batch_size=BATCH_SIZE)

In [ ]:
class SummaryModel(pl.LightningModule):

    def __init__(self):

        super(SummaryModel, self).__init__()

        self.summary_model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):

        x = self.summary_model(input_ids,
                       attention_mask=attention_mask,
                       decoder_attention_mask=decoder_attention_mask,
                       labels=labels
                       )
        
        return x.logits, x.loss

    def step(self, batch, batch_idx, mode="train"):

        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        outputs, loss = self(input_ids, attention_mask, labels_attention_mask, labels)

        self.log(f"{mode}_loss", loss, prog_bar=True, logger=True)

        return loss

    def training_step(self, batch, batch_idx):

        return self.step(batch, batch_idx, mode="train")

    def validation_step(self, batch, batch_idx):

        return self.step(batch, batch_idx, mode="val")

    def test_step(self, batch, batch_idx):

        return self.step(batch, batch_idx, mode="test")

    def configure_optimizers(self):

        return AdamW(self.parameters(), lr=1e-4)



In [ ]:
model = SummaryModel()


In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/checkpoints",
    filename="model-checkpoint",
    monitor="val_loss",
    verbose=True,
    save_top_k=1,
    mode="min"
)

trainer = pl.Trainer(checkpoint_callback=checkpoint_callback,
                     max_epochs=N_EPOCHS,
                     gpus=1,
                     progress_bar_refresh_rate=30)




GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                       | Params
-------------------------------------------------------------
0 | summary_model | T5ForConditionalGeneration | 222 M 
-------------------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Epoch 0, global step 466: val_loss reached 1.44255 (best 1.44255), saving model to "/content/checkpoints/model-checkpoint.ckpt" as top 1


Epoch 1, global step 933: val_loss reached 1.42632 (best 1.42632), saving model to "/content/checkpoints/model-checkpoint.ckpt" as top 1


Epoch 2, global step 1400: val_loss reached 1.42519 (best 1.42519), saving model to "/content/checkpoints/model-checkpoint.ckpt" as top 1


Epoch 3, step 1867: val_loss was not in top 1


Epoch 4, step 2334: val_loss was not in top 1


Epoch 5, step 2801: val_loss was not in top 1


1

In [ ]:
trained_model = SummaryModel.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)

trained_model.freeze()



In [ ]:
def summarize(text):

    text_encoding = tokenizer(
        text,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )

    generated_ids = trained_model.summary_model.generate(
        input_ids=text_encoding["input_ids"],
        attention_mask=text_encoding["attention_mask"],
        max_length=128,
        num_beams=4,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [tokenizer.decode(gen_id, 
                              skip_special_tokens=True, 
                              clean_up_tokenization_spaces=True) for gen_id in generated_ids]

    return " ".join(preds)

        



In [ ]:
test_df.iloc[33]["text"]

'Delhi Chief Minister Arvind Kejriwal is embroiled in a fresh controversy over staying in the house of Khalistan Liberation Front (KLF) activist Gurwinder Singh during his Punjab visit on Saturday.Gurwinder is the former chief of KLF. He is accused of inciting Sikh-Hindu riots during the years when terrorism was at its peak in Punjab. He was even jailed in cases relating to murder and other heinous crimes.However, he was later acquitted and went to England. He is allegedly working for KLF from England.With Aam Aadmi Party (AAP) convener Arvind Kejriwal staying in Gurwinder\'s house in Moga ahead of the February 4 Punjab Assembly elections, the politics in the state is on the boil.AAP\'s rivals - Congress and Akali Dal - have once again repeated their charge against AAP of hobnobbing with anti-Punjab and pro-Khalistan radical forces. They have alleged that AAP is seeking their support and getting funds from them for the Punjab elections.Besides, Hindu organisations have also come out in

In [ ]:
text = test_df.iloc[33]

real = text["summary"]
text = text["text"]



In [ ]:
text

'Delhi Chief Minister Arvind Kejriwal is embroiled in a fresh controversy over staying in the house of Khalistan Liberation Front (KLF) activist Gurwinder Singh during his Punjab visit on Saturday.Gurwinder is the former chief of KLF. He is accused of inciting Sikh-Hindu riots during the years when terrorism was at its peak in Punjab. He was even jailed in cases relating to murder and other heinous crimes.However, he was later acquitted and went to England. He is allegedly working for KLF from England.With Aam Aadmi Party (AAP) convener Arvind Kejriwal staying in Gurwinder\'s house in Moga ahead of the February 4 Punjab Assembly elections, the politics in the state is on the boil.AAP\'s rivals - Congress and Akali Dal - have once again repeated their charge against AAP of hobnobbing with anti-Punjab and pro-Khalistan radical forces. They have alleged that AAP is seeking their support and getting funds from them for the Punjab elections.Besides, Hindu organisations have also come out in

In [ ]:
summary = summarize(text)

In [ ]:
print(summary)

Delhi Chief Minister Arvind Kejriwal has stayed in the house of Khalistan Liber Front (KLF) activist Gurwinder Singh during his Punjab visit on Saturday. The former KLF chief is accused of inciting Sikh-Hindu riots during the years when terrorism was at its peak in Punjab. However, he was later acquitted and went to England.


In [ ]:
n99real

'Delhi CM Arvind Kejriwal has sparked a controversy for staying in the house of alleged ex-militant and Khalistan Liberation Front activist Gurwinder Singh during his Punjab visit on Saturday. Singh, an acquitted Khalistan Commando Force militant, is accused of inciting Sikh-Hindu riots in Punjab and was jailed in various cases including murder. He is reportedly working for KLF from England.'